# 1.合并所有题目和用户

python /data2/liyu/KT/OKT/data_merge.py \
  /data2/liyu/KT/OKT/Project_CodeNet/metadata \
  -o /data2/liyu/KT/OKT/Project_CodeNet/userdata

# 2. 按照代码语言分类

In [1]:
import os
import csv

base_dir = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"  # 根目录：含 user_id.csv
header = None

for fname in os.listdir(base_dir):
    if not (fname.startswith("u") and fname.endswith(".csv")):
        continue  # 跳过非用户文件
    user_csv = os.path.join(base_dir, fname)

    # 读取用户CSV
    with open(user_csv, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        rows = list(reader)
        if not rows:
            continue
        if header is None:
            header = rows[0]
        data_rows = rows[1:]

    if not data_rows:
        # 空用户文件，直接移除
        os.remove(user_csv)
        continue

    # 按语言分组（language 在第5列，索引4）
    lang_to_rows = {}
    for r in data_rows:
        if len(r) <= 4:
            continue
        lang = (r[4] or "Unknown").strip() or "Unknown"
        lang_to_rows.setdefault(lang, []).append(r)

    # 将该用户的记录“转移”到对应语言子目录：userdata/<Language>/<user_id>.csv
    user_id = fname[:-4]
    for lang, lang_rows in lang_to_rows.items():
        lang_dir = os.path.join(base_dir, lang)
        os.makedirs(lang_dir, exist_ok=True)
        out_path = os.path.join(lang_dir, f"{user_id}.csv")
        # 按 date(索引3) 与 submission_id(索引0) 排序，保证稳定
        try:
            lang_rows.sort(key=lambda r: (int(r[3]), r[0]))
        except Exception:
            lang_rows.sort(key=lambda r: (r[3], r[0]))
        with open(out_path, "w", newline="", encoding="utf-8") as out_f:
            w = csv.writer(out_f)
            w.writerow(header)
            w.writerows(lang_rows)

    # 删除根目录下原始的 user_id.csv，实现“转移”
    os.remove(user_csv)

# 3. 为每条提交记录补充上对应的代码

In [ ]:
import os
import csv

def get_code_content(problem_id, submission_id, language, codenet_data_dir):
    """根据problem_id, submission_id, language从Project_CodeNet/data获取代码内容"""
    # 构建文件路径：data/pXXXXX/Language/sXXXXXXXXX.ext
    problem_dir = os.path.join(codenet_data_dir, problem_id)
    if not os.path.exists(problem_dir):
        return ""
    
    # 查找语言目录
    for lang_dir in os.listdir(problem_dir):
        if not os.path.isdir(os.path.join(problem_dir, lang_dir)):
            continue
        # 检查是否匹配语言（考虑映射关系，如C++14->C++）
        if language in lang_dir or lang_dir in language:
            lang_path = os.path.join(problem_dir, lang_dir)
            # 查找提交文件
            for filename in os.listdir(lang_path):
                if filename.startswith(submission_id + "."):
                    file_path = os.path.join(lang_path, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            return f.read()
                    except:
                        try:
                            with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
                                return f.read()
                        except:
                            return ""
    return ""

# 主处理逻辑
userdata_dir = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"
codenet_data_dir = "/data2/liyu/KT/OKT/Project_CodeNet/data"

# 遍历所有语言目录
for lang_dir in os.listdir(userdata_dir):
    lang_path = os.path.join(userdata_dir, lang_dir)
    if not os.path.isdir(lang_path):
        continue
    
    print(f"处理语言: {lang_dir}")
    
    # 遍历该语言下的所有用户CSV
    for user_file in os.listdir(lang_path):
        if not user_file.endswith('.csv'):
            continue
            
        user_csv_path = os.path.join(lang_path, user_file)
        print(f"  处理用户: {user_file}")
        
        # 读取原CSV
        rows = []
        with open(user_csv_path, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            rows = list(reader)
        
        if not rows:
            continue
            
        header = rows[0]
        # 添加新列
        header.append('code_content')
        
        # 处理数据行
        for i in range(1, len(rows)):
            row = rows[i]
            if len(row) < 6:  # 确保有足够的列
                row.extend([''] * (6 - len(row)))
            
            # 提取关键信息：submission_id(0), problem_id(1), language(4)
            submission_id = row[0]
            problem_id = row[1] 
            language = row[4]
            
            # 获取代码内容
            code_content = get_code_content(problem_id, submission_id, language, codenet_data_dir)
            row.append(code_content)
        
        # 写回CSV
        with open(user_csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(rows)

print("完成！所有用户CSV已添加代码内容列")

处理语言: Text
  处理用户: u580402951.csv
  处理用户: u809153881.csv
  处理用户: u424655672.csv
  处理用户: u761320129.csv
  处理用户: u401900157.csv
  处理用户: u823083725.csv
  处理用户: u502709453.csv
  处理用户: u901555384.csv
  处理用户: u825168842.csv
  处理用户: u548842391.csv
  处理用户: u194412908.csv
  处理用户: u176982046.csv
  处理用户: u065125881.csv
  处理用户: u982234424.csv
  处理用户: u008229752.csv
  处理用户: u670220889.csv
  处理用户: u265065462.csv
  处理用户: u495699318.csv
  处理用户: u066337396.csv
  处理用户: u970002663.csv
  处理用户: u898167769.csv
  处理用户: u211681714.csv
  处理用户: u264304509.csv
  处理用户: u076506345.csv
  处理用户: u657913472.csv
  处理用户: u097204018.csv
  处理用户: u906208439.csv
  处理用户: u025432087.csv
  处理用户: u096635831.csv
  处理用户: u090649502.csv
  处理用户: u735089337.csv
  处理用户: u762921614.csv
  处理用户: u330984117.csv
  处理用户: u548163222.csv
  处理用户: u054475353.csv
  处理用户: u876335718.csv
  处理用户: u962966398.csv
  处理用户: u374099594.csv
  处理用户: u350836088.csv
  处理用户: u212437180.csv
  处理用户: u492373312.csv
  处理用户: u075551644.csv
  处理用户: u162773977.csv


In [ ]:
import os
import csv
from tqdm import tqdm

def get_code_content(problem_id, submission_id, language, codenet_data_dir):
    """根据problem_id, submission_id, language从Project_CodeNet/data获取代码内容"""
    # 构建文件路径：data/pXXXXX/Language/sXXXXXXXXX.ext
    problem_dir = os.path.join(codenet_data_dir, problem_id)
    if not os.path.exists(problem_dir):
        return ""
    
    # 直接查找对应的语言目录
    lang_path = os.path.join(problem_dir, language)
    if not os.path.exists(lang_path):
        return ""
    
    # 查找提交文件
    for filename in os.listdir(lang_path):
        if filename.startswith(submission_id + "."):
            file_path = os.path.join(lang_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    return f.read()
            except:
                try:
                    with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
                        return f.read()
                except:
                    return ""
    return ""

# 主处理逻辑 - 只处理指定语言
target_language = "Python"  # 修改这里指定要处理的语言
userdata_dir = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"
codenet_data_dir = "/data2/liyu/KT/OKT/Project_CodeNet/data"

lang_path = os.path.join(userdata_dir, target_language)
if not os.path.exists(lang_path):
    print(f"语言目录不存在: {target_language}")
    exit()

print(f"处理语言: {target_language}")

# 获取所有用户CSV文件列表
user_files = [f for f in os.listdir(lang_path) if f.endswith('.csv')]
print(f"找到 {len(user_files)} 个用户文件")

# 使用tqdm显示进度条
for user_file in tqdm(user_files, desc=f"处理{target_language}用户", unit="用户"):
    user_csv_path = os.path.join(lang_path, user_file)
    
    # 读取原CSV
    rows = []
    with open(user_csv_path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = list(reader)
    
    if not rows:
        continue
        
    header = rows[0]
    # 添加新列
    header.append('code_content')
    
    # 处理数据行
    for i in range(1, len(rows)):
        row = rows[i]
        if len(row) < 6:  # 确保有足够的列
            row.extend([''] * (6 - len(row)))
        
        # 提取关键信息：submission_id(0), problem_id(1), language(4)
        submission_id = row[0]
        problem_id = row[1] 
        language = row[4]
        
        # 获取代码内容
        code_content = get_code_content(problem_id, submission_id, language, codenet_data_dir)
        row.append(code_content)
    
    # 写回CSV
    with open(user_csv_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

print(f"完成！{target_language}语言下所有用户CSV已添加代码内容列")

处理语言: Python
找到 44353 个用户文件


处理Python用户:   0%|          | 15/44353 [00:40<61:55:32,  5.03s/用户]

# 4. 在用户userdata中添加problem information

In [1]:
import os
import csv
import sys
from tqdm import tqdm

# 增加CSV字段大小限制
csv.field_size_limit(1000000)

def load_problem_list(problem_list_path):
    """加载problem_list.csv，返回problem_id到题目信息的映射"""
    problem_info = {}
    with open(problem_list_path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # 读取表头
        for row in reader:
            if len(row) >= 7:  # 确保有足够的列
                problem_id = row[0]
                problem_info[problem_id] = {
                    'name': row[1],
                    'dataset': row[2],
                    'time_limit': row[3],
                    'memory_limit': row[4],
                    'rating': row[5],
                    'tags': row[6],
                    'complexity': row[7] if len(row) > 7 else ''
                }
    return problem_info, header

def add_problem_info_to_user_csv(user_csv_path, problem_info, problem_header):
    """为用户CSV添加题目信息列"""
    # 读取用户CSV
    rows = []
    try:
        with open(user_csv_path, 'r', newline='', encoding='utf-8', errors='ignore') as f:
            reader = csv.reader(f)
            rows = list(reader)
    except Exception as e:
        print(f"读取文件出错 {user_csv_path}: {e}")
        return
    
    if not rows:
        return
    
    header = rows[0]
    # 添加题目信息列（跳过problem_id，因为已经存在）
    new_columns = ['problem_name', 'problem_dataset', 'problem_time_limit', 
                   'problem_memory_limit', 'problem_rating', 'problem_tags', 'problem_complexity']
    header.extend(new_columns)
    
    # 处理数据行
    for i in range(1, len(rows)):
        row = rows[i]
        if len(row) < 2:  # 确保有problem_id列
            continue
            
        problem_id = row[1]  # problem_id在第2列（索引1）
        
        # 获取题目信息
        if problem_id in problem_info:
            info = problem_info[problem_id]
            row.extend([
                info['name'],
                info['dataset'],
                info['time_limit'],
                info['memory_limit'],
                info['rating'],
                info['tags'],
                info['complexity']
            ])
        else:
            # 如果找不到题目信息，填充空值
            row.extend([''] * len(new_columns))
    
    # 写回CSV
    try:
        with open(user_csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(rows)
    except Exception as e:
        print(f"写入文件出错 {user_csv_path}: {e}")

# 主处理逻辑
if len(sys.argv) < 2:
    print("用法: python script.py <target_language>")
    print("例如: python script.py Python")
    sys.exit(1)

# target_language = sys.argv[1]
target_language = "C"
userdata_dir = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"
problem_list_path = "/data2/liyu/KT/OKT/Project_CodeNet/userdata/problem_list.csv"

lang_path = os.path.join(userdata_dir, target_language)
if not os.path.exists(lang_path):
    print(f"语言目录不存在: {target_language}")
    sys.exit(1)

if not os.path.exists(problem_list_path):
    print(f"题目列表文件不存在: {problem_list_path}")
    sys.exit(1)

print(f"处理语言: {target_language}")

# 加载题目信息
print("加载题目信息...")
problem_info, problem_header = load_problem_list(problem_list_path)
print(f"加载了 {len(problem_info)} 个题目信息")

# 获取所有用户CSV文件列表
user_files = [f for f in os.listdir(lang_path) if f.endswith('.csv')]
print(f"找到 {len(user_files)} 个用户文件")

# 使用tqdm显示进度条
for user_file in tqdm(user_files, desc=f"处理{target_language}用户", unit="用户"):
    user_csv_path = os.path.join(lang_path, user_file)
    add_problem_info_to_user_csv(user_csv_path, problem_info, problem_header)

print(f"完成！{target_language}语言下所有用户CSV已添加题目信息列")

处理语言: C
加载题目信息...
加载了 4053 个题目信息
找到 30633 个用户文件


处理C用户:  15%|█▍        | 4494/30633 [00:40<02:27, 177.54用户/s]

读取文件出错 /data2/liyu/KT/OKT/Project_CodeNet/userdata/C/u999854911.csv: line contains NUL


处理C用户:  26%|██▌       | 8020/30633 [01:06<03:07, 120.80用户/s]

读取文件出错 /data2/liyu/KT/OKT/Project_CodeNet/userdata/C/u286936764.csv: line contains NUL


...

读取文件出错 /data2/liyu/KT/OKT/Project_CodeNet/userdata/C/u399107199.csv: line contains NUL


处理C用户:  75%|███████▍  | 22921/30633 [01:43<00:06, 1258.17用户/s]

读取文件出错 /data2/liyu/KT/OKT/Project_CodeNet/userdata/C/u211771669.csv: line contains NUL


处理C用户: 100%|██████████| 30633/30633 [02:04<00:00, 246.83用户/s] 

完成！C语言下所有用户CSV已添加题目信息列


In [3]:
import os
import csv
import sys
from tqdm import tqdm
import re

# 增加CSV字段大小限制
csv.field_size_limit(1000000)

def extract_text_from_html(html_content):
    """从HTML内容中提取纯文本"""
    if not html_content:
        return ""
    
    # 移除HTML标签
    text = re.sub(r'<[^>]+>', '', html_content)
    # 解码HTML实体
    text = text.replace('&lt;', '<').replace('&gt;', '>').replace('&amp;', '&')
    text = text.replace('&quot;', '"').replace('&#39;', "'")
    # 清理多余空白
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def load_problem_descriptions(problem_descriptions_dir):
    """加载所有题目描述HTML文件，返回problem_id到描述内容的映射"""
    problem_descriptions = {}
    
    for filename in os.listdir(problem_descriptions_dir):
        if filename.startswith('p') and filename.endswith('.html'):
            problem_id = filename[:-5]  # 去掉.html后缀
            file_path = os.path.join(problem_descriptions_dir, filename)
            
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    html_content = f.read()
                    # 提取纯文本
                    text_content = extract_text_from_html(html_content)
                    problem_descriptions[problem_id] = text_content
            except Exception as e:
                print(f"读取文件 {filename} 时出错: {e}")
                problem_descriptions[problem_id] = ""
    
    return problem_descriptions

def add_problem_description_to_user_csv(user_csv_path, problem_descriptions):
    """为用户CSV添加题目描述列"""
    # 读取用户CSV，处理NUL字符
    rows = []
    try:
        with open(user_csv_path, 'r', newline='', encoding='utf-8', errors='replace') as f:
            content = f.read()
            # 移除NUL字符
            content = content.replace('\0', '')
            # 使用StringIO来处理内容
            from io import StringIO
            reader = csv.reader(StringIO(content))
            rows = list(reader)
    except Exception as e:
        print(f"读取文件 {user_csv_path} 时出错: {e}")
        return
    
    if not rows:
        return
    
    header = rows[0]
    # 添加题目描述列
    if 'problem_description' not in header:
        header.append('problem_description')
    
    # 处理数据行
    for i in range(1, len(rows)):
        row = rows[i]
        if len(row) < 2:  # 确保有problem_id列
            continue
            
        problem_id = row[1]  # problem_id在第2列（索引1）
        
        # 获取题目描述
        if problem_id in problem_descriptions:
            description = problem_descriptions[problem_id]
        else:
            description = ""
        
        # 如果行长度不够，扩展
        while len(row) < len(header):
            row.append("")
        
        # 更新或添加描述列
        if len(row) == len(header) - 1:
            row.append(description)
        else:
            row[-1] = description
    
    # 写回CSV
    try:
        with open(user_csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(rows)
    except Exception as e:
        print(f"写入文件 {user_csv_path} 时出错: {e}")

# 主处理逻辑
if len(sys.argv) < 2:
    print("用法: python script.py <target_language>")
    print("例如: python script.py Python")
    sys.exit(1)

# target_language = sys.argv[1]

# target_language = "PHP"
# target_language = "Java"
target_language = "C"
userdata_dir = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"
problem_descriptions_dir = "/data2/liyu/KT/OKT/Project_CodeNet/problem_descriptions"

lang_path = os.path.join(userdata_dir, target_language)
if not os.path.exists(lang_path):
    print(f"语言目录不存在: {target_language}")
    sys.exit(1)

if not os.path.exists(problem_descriptions_dir):
    print(f"题目描述目录不存在: {problem_descriptions_dir}")
    sys.exit(1)

print(f"处理语言: {target_language}")

# 加载题目描述
print("加载题目描述...")
problem_descriptions = load_problem_descriptions(problem_descriptions_dir)
print(f"加载了 {len(problem_descriptions)} 个题目描述")

# 获取所有用户CSV文件列表
user_files = [f for f in os.listdir(lang_path) if f.endswith('.csv')]
print(f"找到 {len(user_files)} 个用户文件")

# 使用tqdm显示进度条
for user_file in tqdm(user_files, desc=f"处理{target_language}用户", unit="用户"):
    user_csv_path = os.path.join(lang_path, user_file)
    add_problem_description_to_user_csv(user_csv_path, problem_descriptions)

print(f"完成！{target_language}语言下所有用户CSV已添加题目描述列")

处理语言: C
加载题目描述...
加载了 3999 个题目描述
找到 30633 个用户文件


处理C用户: 100%|██████████| 30633/30633 [01:46<00:00, 286.93用户/s] 

完成！C语言下所有用户CSV已添加题目描述列


# 5. 统计不同代码语言下用户数量

In [1]:
#!/usr/bin/env python3
"""
分析 Project_CodeNet userdata 文件夹下每个代码语言的用户数量
"""

import os
import pandas as pd
from collections import defaultdict

def analyze_userdata_languages(userdata_path):
    """
    分析 userdata 文件夹下每个语言文件夹的CSV文件数量
    
    Args:
        userdata_path (str): userdata 文件夹路径
    
    Returns:
        dict: 语言名称到CSV文件数量的映射
    """
    language_counts = {}
    
    # 获取所有语言文件夹
    language_dirs = [d for d in os.listdir(userdata_path) 
                    if os.path.isdir(os.path.join(userdata_path, d))]
    
    print(f"发现 {len(language_dirs)} 个语言文件夹")
    print("=" * 50)
    
    total_users = 0
    
    for lang_dir in sorted(language_dirs):
        lang_path = os.path.join(userdata_path, lang_dir)
        
        # 统计该语言文件夹下的CSV文件数量
        csv_files = [f for f in os.listdir(lang_path) if f.endswith('.csv')]
        csv_count = len(csv_files)
        
        language_counts[lang_dir] = csv_count
        total_users += csv_count
        
        print(f"{lang_dir:15} : {csv_count:6} 个用户")
    
    print("=" * 50)
    print(f"总计用户数: {total_users}")
    
    return language_counts

def create_summary_dataframe(language_counts):
    """
    创建汇总的DataFrame
    
    Args:
        language_counts (dict): 语言到用户数的映射
    
    Returns:
        pd.DataFrame: 汇总数据
    """
    # 按用户数降序排列
    sorted_languages = sorted(language_counts.items(), key=lambda x: x[1], reverse=True)
    
    df = pd.DataFrame(sorted_languages, columns=['编程语言', '用户数量'])
    df['排名'] = range(1, len(df) + 1)
    
    # 计算百分比
    total_users = df['用户数量'].sum()
    df['占比(%)'] = (df['用户数量'] / total_users * 100).round(2)
    
    return df

def main():
    userdata_path = "/data2/liyu/KT/OKT/Project_CodeNet/userdata"
    
    print("Project_CodeNet 用户数据分析")
    print("=" * 50)
    
    # 分析各语言用户数量
    language_counts = analyze_userdata_languages(userdata_path)
    
    print("\n详细统计表格:")
    print("=" * 80)
    
    # 创建汇总DataFrame
    summary_df = create_summary_dataframe(language_counts)
    
    # 显示前20名
    print("前20名编程语言:")
    print(summary_df.head(20).to_string(index=False))
    
    print(f"\n完整统计信息:")
    print(f"- 总语言数: {len(language_counts)}")
    print(f"- 总用户数: {summary_df['用户数量'].sum()}")
    print(f"- 平均每语言用户数: {summary_df['用户数量'].mean():.1f}")
    print(f"- 用户数最多的语言: {summary_df.iloc[0]['编程语言']} ({summary_df.iloc[0]['用户数量']} 用户)")
    print(f"- 用户数最少的语言: {summary_df.iloc[-1]['编程语言']} ({summary_df.iloc[-1]['用户数量']} 用户)")
    
    # 保存结果到CSV
    output_file = "/data2/liyu/KT/OKT/language_user_analysis.csv"
    summary_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n结果已保存到: {output_file}")
    
    return summary_df

if __name__ == "__main__":
    result_df = main()

Project_CodeNet 用户数据分析
发现 55 个语言文件夹
Ada             :     10 个用户
Awk             :    182 个用户
Bash            :    542 个用户
Bf              :    338 个用户
C               :  30633 个用户
C#              :   4128 个用户
C++             : 102866 个用户
COBOL           :     41 个用户
Ceylon          :     11 个用户
Clojure         :     93 个用户
Crystal         :    110 个用户
Cython          :    275 个用户
D               :    325 个用户
Dart            :     29 个用户
Dash            :      2 个用户
Elixir          :     22 个用户
Erlang          :      4 个用户
F#              :    131 个用户
Forth           :      7 个用户
Fortran         :    250 个用户
Go              :   2473 个用户
Haskell         :   1213 个用户
Haxe            :     15 个用户
Java            :  16327 个用户
JavaScript      :   2560 个用户
Julia           :    275 个用户
Kotlin          :   1034 个用户
Lisp            :    176 个用户
Lua             :     89 个用户
MoonScript      :     14 个用户
Nim             :    211 个用户
OCaml           :    245 个用户
Objective-C     :     46 个用户
Octave 